In [1]:

# NECESSARY IMPORTS

from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
import tensorflow as tf
import math 
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model



In [2]:
# NEURAL NETWORK DEFINITION

class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    
    self.encoder = tf.keras.Sequential([
      layers.Dense(32, activation="relu", input_shape=(45,)), 
      layers.Dense(16, activation="relu"),
      layers.Dense(8, activation="relu")])

    self.decoder = tf.keras.Sequential([
      layers.Dense(16, activation="relu", input_shape=(8,)),
      layers.Dense(32, activation="relu"),
      layers.Dense(45, activation="sigmoid")]) #NUMBER OF INPUT FEATURES

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded






In [3]:
# READING THE DATASET

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df1 = pd.read_csv("/home/francesc/Escritorio/dataset_kaggle/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2=pd.read_csv("/home/francesc/Escritorio/dataset_kaggle/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3=pd.read_csv("/home/francesc/Escritorio/dataset_kaggle/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv")
df4=pd.read_csv("/home/francesc/Escritorio/dataset_kaggle/MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv")
df5=pd.read_csv("/home/francesc/Escritorio/dataset_kaggle/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
#df6=pd.read_csv("/home/francesc/Escritorio/dataset_kaggle/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
#df7=pd.read_csv("/home/francesc/Escritorio/dataset_kaggle/MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv")
#df8=pd.read_csv("/home/francesc/Escritorio/dataset_kaggle/MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv")

In [4]:
# CONCATENING ALL THE DATASETS FROM EVERY DAY

df = pd.concat([df1,df2])
del df1,df2
df = pd.concat([df,df3])
del df3
df = pd.concat([df,df4])
del df4
df = pd.concat([df,df5])
del df5
#df = pd.concat([df,df6]) # its not necessary to use them all...
#del df6
#df = pd.concat([df,df7])
#del df7
#df = pd.concat([df,df8])
#del df8
nRow, nCol = df.shape


In [5]:

df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.666700,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109,109,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,9174.311927,9174.311927,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52,52,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,19230.769230,19230.769230,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34,34,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,29411.764710,29411.764710,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,31,329,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.666700,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [6]:
# TO SAVE MEMORY LET'S CONVERT ALL TO INT32 OR FLOAT 32 (NOT INT 64 OR FLOAT 64)

integer = []
f = []
for i in df.columns[:-1]:
    if df[i].dtype == "int64": integer.append(i)
    else : f.append(i)

df[integer] = df[integer].astype("int32")
df[f] = df[f].astype("float32")

In [7]:
# DROP VARIABLES THAT HAVE A CONSTANT VALUE

one_variable_list = []
for i in df.columns:
    if df[i].value_counts().nunique() < 2:
        one_variable_list.append(i)
df.drop(one_variable_list,axis=1,inplace=True)
df.columns =  df.columns.str.strip()

In [8]:
df = df.dropna()


In [9]:
# DROP ROWS THAT HAVE INF VALUES

df = df[~df.isin([np.inf]).any(1)]

In [10]:
# FEATURE REDUCTION: DROP HIGH CORRELATED FEATURES

corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
df =  df.drop(to_drop, axis=1)


In [11]:
df = df.drop(["Destination Port"],axis=1)

In [12]:
df.shape

(1520594, 46)

In [13]:
# AFTER PREPROCESSING...

df["Label"].value_counts()

BENIGN          1231773
PortScan         158804
DDoS             128025
Bot                1956
Infiltration         36
Name: Label, dtype: int64

In [14]:
df = df[df["Label"] == 'BENIGN'] # we want just benign records to train autoencoders...

In [15]:
df["Label"].value_counts()

BENIGN    1231773
Name: Label, dtype: int64

In [16]:
df.shape

(1231773, 46)

In [17]:
#AQUI-1

In [18]:
x = df.drop(["Label"],axis=1)
y = df["Label"]

In [19]:
x = x.sample(frac = 1)

In [23]:

list_of_datasets = []

n_workers = 4
#size_dataset = int(1000000/n_workers)

for i in range(n_workers): 
    #start_range = i*size_dataset
    #end_range = start_range + size_dataset
    #list_of_datasets.append(x[start_range:end_range])
    list_of_datasets.append(x[0:1000000].sample(frac = 0.10))

X_test= x[1000000:1005000] 

In [24]:

# PRINTING INFO...

for i in range(n_workers): 
    print(list_of_datasets[i].shape)
    print(type(list_of_datasets[i]))
    
#print(x1.shape)
#print(x2.shape) 
print(X_test.shape)




(100000, 45)
<class 'pandas.core.frame.DataFrame'>
(100000, 45)
<class 'pandas.core.frame.DataFrame'>
(100000, 45)
<class 'pandas.core.frame.DataFrame'>
(100000, 45)
<class 'pandas.core.frame.DataFrame'>
(5000, 45)


In [25]:
# SPLITTING DATASET INTO TRAIN AND TEST

list_of_train_set = []
list_of_val_set = []

for i in range(n_workers): 
    X_train, X_val = train_test_split(list_of_datasets[i],test_size=0.10,random_state=42)
    list_of_train_set.append(X_train)
    list_of_val_set.append(X_val)


print(len(list_of_train_set))
print(len(list_of_val_set))


4
4


In [26]:

# STANDARIZING...

list_of_train_set_esc = []
list_of_val_set_esc = []

for i in range(n_workers): 
    X_train = list_of_train_set[i]
    X_val = list_of_val_set[i]
    
    scaler = StandardScaler()
    
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    
    X_train = pd.DataFrame(X_train)
    X_val = pd.DataFrame(X_val)
    
    list_of_train_set_esc.append(X_train)
    list_of_val_set_esc.append(X_val)
    
    if i==0:
        X_test = scaler.transform(X_test) # with the first one for example...
        X_test = pd.DataFrame(X_test)







In [27]:
# THIS IS TO USE IT LATER WHEN NOT USING THE SIMULATOR BUT THE REAL PROGRAM...

X_test.to_csv('/home/francesc/Escritorio/federated-learning-network-main/server/records_test.csv',index=False)

port_number=5001

for i in range(n_workers): 

    filename=str(port_number)
    
    list_of_train_set_esc[i].to_csv('/home/francesc/Escritorio/federated-learning-network-main/client/datasets/'+filename+'_train.csv',index=False)
    list_of_val_set_esc[i].to_csv('/home/francesc/Escritorio/federated-learning-network-main/client/datasets/'+filename+'_val.csv',index=False)

    port_number = port_number+1
   


In [28]:

# WE NEED NUMPY ARRAYS NOT PANDAS DATAFRAMES...


list_of_train_np = []
list_of_val_np = []

for i in range(n_workers): 
    X_train = list_of_train_set_esc[i].to_numpy()
    X_val = list_of_val_set_esc[i].to_numpy()
    
    list_of_train_np.append(X_train)
    list_of_val_np.append(X_val)
    


X_test = X_test.to_numpy()




In [29]:
#AQUI-2

In [30]:

## FIRST TRADITIONAL MACHINE LEARNING TO COMPARE IT LATER WITH FEDERATED MACHINE LEARNING (IN TERMS OF TEST LOSS)


In [33]:

# TRADITIONAL MACHINE LEARNING

list_of_autoencoders = []

#WORKERS
for i in range (n_workers):
    autoencoder = AnomalyDetector()
    optimizer = Adam(learning_rate=0.001)
    autoencoder.compile(optimizer=optimizer, loss='mse') 
    list_of_autoencoders.append(autoencoder)



In [34]:
test_losses = []

for i in range (n_workers): 
    list_of_autoencoders[i].fit(list_of_train_np[i], list_of_train_np[i], epochs=10, batch_size=256, validation_data=(list_of_val_np[i], list_of_val_np[i]), shuffle=True,verbose=0)  

    print(i)
    
    reconstructions = list_of_autoencoders[i].predict(X_test)
    test_loss = np.mean(tf.keras.losses.mse(reconstructions, X_test))
    
    test_losses.append(test_loss)
    #time.sleep(2)
    
print(sum(test_losses)/n_workers)    
        
    

0
1
2
3
1.4754409178211305


In [35]:

# FEDERATED MACHINE LEARNING 

list_of_autoencoders = []

#WORKERS
for i in range (n_workers):
    autoencoder = AnomalyDetector()
    optimizer = Adam(learning_rate=0.001)
    autoencoder.compile(optimizer=optimizer, loss='mse') 
    list_of_autoencoders.append(autoencoder)

#SERVER
autoencoder_server = AnomalyDetector()
optimizer_server = Adam(learning_rate=0.001)
autoencoder_server.compile(optimizer=optimizer_server, loss='mse')


In [ ]:

import sys

for i in range(1000): # represents the number of virtual rounds...

    print("Ronda {}\n\n".format(i)) 
    
    received_weights = []
    for j in range(n_workers):
        
        list_of_autoencoders[j].fit(list_of_train_np[j], list_of_train_np[j], epochs=1, batch_size=256, validation_data=(list_of_val_np[j], list_of_val_np[j]), shuffle=True,verbose=0)  
        received_weights.append(list_of_autoencoders[j].get_weights())
        
    new_weights = np.stack(received_weights).mean(0)    
    
    for j in range(n_workers):
        list_of_autoencoders[j].set_weights(new_weights)
        
        
        
    
    autoencoder_server.set_weights(new_weights)
        
    print("\n\n\n\n\n")
        
    reconstructions = autoencoder_server.predict(X_test)
    test_loss = np.mean(tf.keras.losses.mse(reconstructions, X_test))
        
    print("Test loss {}.".format(test_loss))
    
    print("\n\n\n\n\n")
    
    ##########################################
    
    
    time.sleep(5) #let the server take a breath... if not the notebook kernel crashes...
    
    
    sys.stdout.flush() 
    
    
    

Ronda 0




/home/francesc/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)








Test loss 1.7552720479212316.






Ronda 1








Test loss 1.5620165327233864.






Ronda 2








Test loss 1.5412380616171932.






Ronda 3








Test loss 1.532450389509947.






Ronda 4








Test loss 1.5210746734979925.






Ronda 5








Test loss 1.5196994802999642.






Ronda 6








Test loss 1.5188508983259024.






Ronda 7








Test loss 1.5182625658980522.






Ronda 8








Test loss 1.517777512861957.






Ronda 9








Test loss 1.5171884809592158.






Ronda 10








Test loss 1.5160722762732164.






Ronda 11








Test loss 1.5153483902687035.






Ronda 12








Test loss 1.5132585838453576.






Ronda 13








Test loss 1.5125372119370557.






Ronda 14








Test loss 1.5122065986754953.






Ronda 15








Test loss 1.5116492069809178.






Ronda 16








Test loss 1.511159020217081.






Ronda 17








Test loss 1.5096397136306465.






Ronda 18








Test loss 1.5100042569294467.






Ronda 19









In [37]:
#weights_1 = autoencoder.get_weights()
#weights_2 = autoencoder2.get_weights()

#print(type(weights_1)) #list
#print(type(weights_2)) #list

#print((weights_1)[0][0])
#print((weights_2)[0][0])

#received_weights = [weights_1,weights_2]

#new_weights = np.stack(received_weights).mean(0)

#print((new_weights)[0][0])

#autoencoder.set_weights(new_weights)
#autoencoder2.set_weights(new_weights)

[-0.11712989  0.00963387 -0.1758296   0.14854908  0.24154207  0.22534415
  0.04534206 -0.23300013  0.02302489 -0.03373799  0.23524949  0.00304216
  0.00050646  0.15862808  0.04321539  0.03914532 -0.09775107  0.1762841
 -0.06999543  0.11604726  0.06951264 -0.20754205  0.06162888 -0.15184616
  0.00615993 -0.05692454  0.02954265 -0.03657217 -0.23452173 -0.2082519
 -0.1292492  -0.1597511 ]
[ 0.19741076  0.21361053  0.19305575 -0.2526198  -0.2790817   0.18229607
 -0.12110859  0.1060144   0.05004472  0.19379404  0.0543189   0.16939288
  0.1673783   0.18015245  0.20483586 -0.04577571 -0.20132357 -0.00862426
 -0.02353388 -0.18426485 -0.2496098   0.14043531  0.11001319  0.00773272
  0.22503033  0.24920353  0.19090801 -0.10651802 -0.11078694 -0.02468488
 -0.26296148 -0.12632658]
[ 0.04014044  0.1116222   0.00861307 -0.05203536 -0.01876982  0.20382011
 -0.03788327 -0.06349286  0.0365348   0.08002803  0.1447842   0.08621752
  0.08394238  0.16939026  0.12402563 -0.0033152  -0.14953732  0.08382992
 